In [1]:
import cv2
import joblib
import matplotlib.pyplot as plt
from PIL import Image
from collections import Counter
import numpy as np

In [17]:
import json
from pathlib import Path

def load_json(fp):
    with open(fp, 'r') as f:
        return json.load(f)


def dump_json(obj, fp):
    with open(fp, 'w') as f:
        json.dump(obj, f)


dataset_dir = Path('/Users/user/Downloads/solaris_damage_0_4_3')
img_dir = dataset_dir / 'images'
anns = load_json(dataset_dir / 'train_damage_coco.json')

id_to_filename = {ann['id']: ann['file_name'] for ann in anns['images']}

all_img_ids = set(ann['id'] for ann in anns['images'])
ann_img_ids = set(ann['image_id'] for ann in anns['annotations'])
len(all_img_ids - ann_img_ids)

1532

In [18]:
np.mean([(img_dir / Path(id_to_filename[ann['image_id']]).name).exists() for ann in anns['annotations']])

1.0

In [7]:
img_dir / Path(ann['file_name']).name

PosixPath('/Users/user/Downloads/solaris_damage_0_4_3/images/837010425_ph-1612345459140.jpg')

In [ ]:
filename_to_entropy = joblib.load('/Users/user/Downloads/filename_to_entropy.pkl')

In [90]:
from typing import Optional

def remove_image_compression_glitch(
        im: np.array,  # RGB [height, width, channels]
        filter_rate: Optional[float] = None,  # пиксели с какой долей от фото какие пиксели даже не рассматриваем
    ):

    im = np.array(im)[..., 0]
    colors_counter = Counter(im.flatten())
    morphed_im = np.zeros_like(im)

    for color, count in colors_counter.items():
        color_rate = count / im.size
        if filter_rate is not None and color_rate < filter_rate:
            continue

        color_mask = (im == color).astype('uint8')
        kernel = np.ones((3, 3), np.uint8)
        color_mask_morph = cv2.morphologyEx(
            color_mask.copy(),
            cv2.MORPH_OPEN,
            kernel,
        )

        color_mask_morph = cv2.morphologyEx(
            color_mask_morph.copy(),
            cv2.MORPH_DILATE,
            kernel,
        )

        morphed_im[color_mask_morph.astype('bool')] = color

    zero_channel = np.zeros_like(morphed_im)
    morphed_im_rgb = np.stack([morphed_im, zero_channel, zero_channel]).transpose((1, 2, 0))
    return morphed_im_rgb

im = Image.open('/Users/user/Downloads/760093081_ph-1594116281881_new.png').convert('RGB')
morphed_im = remove_image_compression_glitch(im)
Image.fromarray(morphed_im).save('/Users/user/Downloads/760093081_ph-1594116281881_no_gap.png')

array([ 0, 10, 28, 79, 83], dtype=uint8)